findspark: Spark' ı Python Script' lerinde kullanbilmeyi sağlayan yapı.

SparkContext is the entry point to any spark functionality. When we run any Spark application, a driver program starts, which has the main function and your SparkContext gets initiated here. The driver program then runs the operations inside the executors on worker nodes. 

SparkContext uses Py4J to launch a JVM and creates a JavaSparkContext [[Tutorials Points](https://www.tutorialspoint.com/pyspark/pyspark_sparkcontext.htm)]

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext
sc = SparkContext(master="local", appName="firtsapp")
# parametre
# master: It is the URL of the cluster it connects to. (anlamadım)

# Bu hücreyi bir kez daha çalıştırsanız şöyle bir hata alıcaksınız:
# ValueError: Cannot run multiple SparkContexts at once

Dizindeki README.md dosyasının içinde `a` ve `b` karakteri geçen satırların sayısını buluyoruz.

In [4]:
file = sc.textFile("README.md").cache()
rowA = file.filter(lambda row: 'a' in row).count()
rowB =file.filter(lambda row: 'b' in row).count()
print(f"Lines with a: {rowA}, lines with b: {rowB}")

Lines with a: 53, lines with b: 26


##  PySpark RDD

RDD stands for Resilient Distributed Dataset, these are the elements that run and operate on multiple nodes to do parallel processing on a cluster.

RDDs are immutable elements, which means once you create an RDD you cannot change it. RDDs are fault tolerant as well, hence in case of any failure, they recover automatically.

To apply operations on these RDD's, there are two ways −
- Transformation and
- Action

Transformation − These are the operations, which are applied on a RDD to create a new RDD. Filter, groupBy and map are the examples of transformations.

Action − These are the operations that are applied on RDD, which instructs Spark to perform computation and send the result back to the driver.

To apply any operation in PySpark, we need to create a PySpark RDD first. 

In [5]:
arr = [
    "scala", 
    "java", 
    "hadoop", 
    "spark", 
    "akka",
    "spark vs hadoop", 
    "pyspark",
    "pyspark and spark"
]
words = sc.parallelize(arr) # RDD nesnesi oluşturur.

In [12]:
# count(): RDD' de bulunan element' lerin sayısını döndürür.
words.count()
print(count)
print("type:", type(count))

8
type: <class 'int'>


In [13]:
# collect(): RDD' deki bütün element' leri döndürür
collect = words.collect() # RDD to Array
print(collect)
print("type: ", type(collect))

['scala', 'java', 'hadoop', 'spark', 'akka', 'spark vs hadoop', 'pyspark', 'pyspark and spark']
type:  <class 'list'>


In [16]:
# foreach(): foreach fonksiyonuna parametre olarak verilen fonksiyonu RDD içindeki bütün elemanlara uygular
def f(x):
    print("Merhaba dünya", x)
words.foreach(f) # Çalışmaıyor. Neden bilmiyorum.

In [17]:
# filter(): Şartı sağlayan elemanlardan yeni bir RDD oluşturur
filtered = words.filter(lambda x: 'spark' in x)
filtered.collect()

['spark', 'spark vs hadoop', 'pyspark', 'pyspark and spark']

In [19]:
# map(): RDD' deki her bir element' e aynı işlemi uygulayarak yeni bir RDD döndürür.
words_map = words.map(lambda x: (x, 1))
words_map.collect()

[('scala', 1),
 ('java', 1),
 ('hadoop', 1),
 ('spark', 1),
 ('akka', 1),
 ('spark vs hadoop', 1),
 ('pyspark', 1),
 ('pyspark and spark', 1)]

In [7]:
# reduce(): Kendisine parametre olarak verilen fonksiyonu bütün elemanlar üzerinde gerçekleştirir. Map fonksiyonundan farkı sanırım şu; map eleman birleştirme vb. gibi işlemler yapmıyor. Bunun adında anlaşılacağı üzere reduce azaltıyor, küçültüyor.
from operator import add
nums = sc.parallelize([1, 2, 3, 4, 5])
print(nums.reduce(add))
print(words.reduce(add))

def my_add(a, b):
    return f'{a}-{b}'

words_concat = words.reduce(my_add)
print(words_concat)
print("words_concat:", type(words_concat))
print(nums.reduce(my_add))

# 2' den fazla parametre alan fonksiyonları çalıştırmıyor sanırım.
#def my_add_2(a, b, c):
#    return f'{a}-{b}-{c}'

# print(words.reduce(my_add_2))

15
scalajavahadoopsparkakkaspark vs hadooppysparkpyspark and spark
scala-java-hadoop-spark-akka-spark vs hadoop-pyspark-pyspark and spark
words_concat: <class 'str'>
1-2-3-4-5


In [14]:
# join(): İki farklı RDD' nin key' lerini baz alarak value değerlerini birleştirir. RDD_A içindeki eleman (k, v1), RDD_2 içindeki eleman (k, v2) ise onları (k, (v1, v2)) şekilinde birleştiriyor. Peki böyle bir şey neden var? Peki neden (k, v) şeklinde olmak zorunda?
x = sc.parallelize([("spark", 1), ("hadoop", 4), ("dene", 4)])
y = sc.parallelize([("spark", 2), ("hadoop", 5), ('spark', 16, 17), ("naber", 4), ['spark', 100]])
# , {'spark': 200} koydum y' ye hata verdi.
joined = x.join(y)
print(joined.collect())

joined = y.join(x)
print(joined.collect())

[('hadoop', (4, 5)), ('spark', (1, 2)), ('spark', (1, 16)), ('spark', (1, 100))]
[('hadoop', (5, 4)), ('spark', (2, 1)), ('spark', (16, 1)), ('spark', (100, 1))]


In [25]:
# cache(): Persist(kalıcı) this RDD with the default storage level (MEMORY_ONLY).
# işlemlerin çok daha hızlı olması içindir. Belki Spark işlemi gerçekleştirdikten sonra RAM' den siliyor. Bu fonksiyon sayesinde verileri RAM' de kalıcı yapıyor

print(words.persist().is_cached)
# persist(): fonksiyonu' da kalıcı yapıyor. Fonksiyonun açıklamasını oku.
# is_cahched RDD nesnesine ait bir değişken

print(nums.is_cached)
print(nums.persist().is_cached)
print(nums.unpersist().is_cached)



True
False
True
False
